In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Irritable_bowel_syndrome_(IBS)/GSE36701'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression analysis of rectal mucosa in chronic irritable bowel syndrome (IBS) compared to healthy volunteers (HV)"
!Series_summary	"An investigation of gene expression changes in rectal biopsies from donors with IBS compared to controls to begin to understand this complex syndrome. To further investigate differences between IBS groups (constipation and diarrhoea predominant) (part1) and how IBS relates to bacterial infection (part2) with biopsies taken 6 months after Campylobacter jejuni infection."
!Series_overall_design	"Part1: 18 Constipation predominant IBS subjects (IBS-C) and 27 diarrhoea predominant IBS subjects (IBS-D) compared to 21 healthy volunteers (HV)."
!Series_overall_design	"Part2: 21 Campylobacter jejuni infection (PIBD, PIBS, PINIBS) compared to 19 healthy volunteers (HV)."
!Series_overall_design	"PIBD = post Campylobacter infection with IBS (within 6 months)"
!Series_overall_design	"PIBS = post infection IBS (unknown time 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Based on the series background and dataset information
is_gene_available = True

# Determining the rows for trait, age, and gender
# Looking at the Sample Characteristics Dictionary
# trait_row corresponds to the "disease" information, which is row 3
trait_row = 3

# No information regarding the "age" in the Sample Characteristics Dictionary
age_row = None

# gender_row corresponds to the "gender" information, which is row 1
gender_row = 1

# Function to convert trait values
def convert_trait(value):
    trait_map = {
        "HV": 0,
        "IBS-D": 1,
        "IBS-C": 1,
        "PIBD": 1,
        "PINIBS": 0,
        "PIBS": 1
    }
    key, val = value.split(": ")
    return trait_map.get(val, None)

# No conversion function for age as age_row is None
convert_age = None

# Function to convert gender values
def convert_gender(value):
    gender_map = {
        "M": 1,
        "F": 0
    }
    key, val = value.split(": ")
    return gender_map.get(val, None)

# Save cohort information
save_cohort_info('GSE36701', './preprocessed/Irritable_bowel_syndrome_(IBS)/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Irritable_bowel_syndrome_(IBS)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Irritable_bowel_syndrome_(IBS)/trait_data/GSE36701.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM899034': [0, 1], 'GSM899035': [0, 1], 'GSM899036': [0, 1], 'GSM899037': [0, 1], 'GSM899038': [0, 0], 'GSM899039': [0, 0], 'GSM899040': [0, 0], 'GSM899041': [0, 1], 'GSM899042': [0, 1], 'GSM899043': [0, 1], 'GSM899044': [0, 1], 'GSM899045': [0, 0], 'GSM899046': [0, 0], 'GSM899047': [0, 0], 'GSM899048': [0, 0], 'GSM899049': [0, 1], 'GSM899050': [0, 1], 'GSM899051': [0, 0], 'GSM899052': [0, 0], 'GSM899053': [0, 0], 'GSM899054': [0, 0], 'GSM899055': [0, 0], 'GSM899056': [0, 0], 'GSM899057': [0, 1], 'GSM899058': [0, 1], 'GSM899059': [0, 1], 'GSM899060': [0, 1], 'GSM899061': [0, 0], 'GSM899062': [0, 0], 'GSM899063': [0, 0], 'GSM899064': [0, 0], 'GSM899065': [0, 0], 'GSM899066': [0, 0], 'GSM899067': [0, 0], 'GSM899068': [0, 1], 'GSM899069': [0, 1], 'GSM899070': [0, 0], 'GSM899071': [0, 0], 'GSM899072': [0, 0], 'GSM899073': [0, 0], 'GSM899074': [1, 0], 'GSM899075': [1, 0], 'GSM899076': [1, 1], 'GSM899077': [1, 1], 'GSM899078': [1, 0], 'GSM899079': [1, 0], 'GSM899080': [1, 0], 'GSM899081':

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. When analyzing a gene expression dataset, we need to map some identifiers of genes to actual gene symbols.
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Irritable_bowel_syndrome_(IBS)/gene_data/GSE36701.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Irritable_bowel_syndrome_(IBS)')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE36701', './preprocessed/Irritable_bowel_syndrome_(IBS)/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Irritable_bowel_syndrome_(IBS)/GSE36701.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Irritable_bowel_syndrome_(IBS)', the least common label is '0.0' with 91 occurrences. This represents 41.18% of the dataset.
The distribution of the feature 'Irritable_bowel_syndrome_(IBS)' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 81 occurrences. This represents 36.65% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

